In [18]:
#global path
global_drive_path="/content/drive/MyDrive/ForeCache/interactions/"
          

In [19]:
import os
import pandas as pd
import json
from pandas.io.json import json_normalize
from tqdm import tqdm


#for reward
import time
import datetime

pd.options.mode.chained_assignment = None  # default='warn'


In [20]:
def clean_files(filePath,fileName):

  lastline = None
 # filePath="/content/drive/Shareddrives/Colearning-Exploration/Dataset/forecache_interactions_2016/forecache-interaction-analysis/forecache-interaction-analysis/interactions/taskname_ndsi-2d-subtask_userid_3abeecbe-327a-441e-be2a-0dd3763c1d45.txt"
  with open(filePath,"r") as f:
      lineList = f.readlines()
      lastline=lineList[-1]
  #with open(filePath,"r") as f, open(global_drive_path+"cleanInteractions/cleanInteractions-ndsi-2d-task/"+(str(fileName)+".json"),"w") as g:
  with open(filePath,"r") as f, open(global_drive_path+"cleanInteractions/cleanInteractions-ndsi-3d-task/"+(str(fileName)+".json"),"w") as g:
  #os.makedirs(os.path.dirname(global_drive_path+"cleanInteractions/cleanInteractions-rgb-hurricane-3d-task/"+(str(fileName)+".json")))
 # with open(filePath,"r") as f, open(global_drive_path+"cleanInteractions/cleanInteractions-rgb-hurricane-3d-task/"+(str(fileName)+".json"),"w+") as g:
   
      for i,line in enumerate(f,0):
          if i == 0:
              line = "["+str(line)+","
              g.write(line)
          elif line == lastline:            
              g.write(line)
              g.write("]")
          else:
              line = str(line)+","
              g.write(line)
          

In [21]:
#directory =(str(global_drive_path) +"/ndsi-2d-task")
directory =(str(global_drive_path) +"/ndsi-3d-task")
#directory =(str(global_drive_path) +"/rgb-hurricane-3d-task")
failed_files=[]
for filename in os.listdir(directory):
  try:
    if filename.endswith(".txt"):
        
        filePath=(os.path.join(directory, filename))
        print(filename)
        name=os.path.splitext(filename)[0]
        clean_files(filePath,name)
    else:
        continue
  except OSError as err:
    print("OS error: {0}".format(err))
    failed_files.append(filename)
  except UnicodeDecodeError:
    print("Unicode Error")
    failed_files.append(filename)
  except BaseException as err:
    failed_files.append(filename)
    
    raise


taskname_ndsi-3d-task_userid_8b544d24-3274-4bb0-9719-fd2bccc87b02.txt
taskname_ndsi-3d-task_userid_bda49380-37ad-41c5-a109-7fa198a7691a.txt
taskname_ndsi-3d-task_userid_954edb7c-4eae-47ab-9338-5c5c7eccac2d.txt
taskname_ndsi-3d-task_userid_e4221e48-0930-40db-af3c-f4de1ed87582.txt
taskname_ndsi-3d-task_userid_d6555293-35c7-4c27-b522-38d67d2d1b1a.txt
taskname_ndsi-3d-task_userid_8bf5b71b-dfd4-403f-8791-06427b26ccaf.txt
taskname_ndsi-3d-task_userid_7d014c75-453b-4188-9e4e-fb086984896a.txt
taskname_ndsi-3d-task_userid_ac5b65d7-f260-4534-9fae-d998b726ed32.txt
taskname_ndsi-3d-task_userid_3abeecbe-327a-441e-be2a-0dd3763c1d45.txt
taskname_ndsi-3d-task_userid_ff56863b-0710-4a58-ad22-4bf2889c9bc0.txt
taskname_ndsi-3d-task_userid_733a1ac5-0b01-485e-9b29-ac33932aa240.txt
taskname_ndsi-3d-task_userid_a6aab5f5-fdb6-41df-9fc6-221d70f8c6e8.txt
taskname_ndsi-3d-task_userid_93be0a02-3f7a-46c9-b526-aa956ed00856.txt
taskname_ndsi-3d-task_userid_82316e37-1117-4663-84b4-ddb6455c83b2.txt
taskname_ndsi-3d-tas

In [22]:
directory =(str(global_drive_path) +"/cleanInteractions//cleanInteractions-ndsi-3d-task")
#directory =(str(global_drive_path) +"/cleanInteractions//cleanInteractions-ndsi-2d-task")
#directory =(str(global_drive_path) +"/cleanInteractions//cleanInteractions-rgb-hurricane-3d-task")
fileNames=[]
for filename in os.listdir(directory):
    if filename.endswith(".json"):
        filePath=(os.path.join(directory, filename))
        fileNames.append(filename)
    else:
        continue

In [23]:
import numpy as np
import matplotlib.pyplot as plt
def angle_between(p1, p2):
    ang1 = np.arctan2(*p1[::-1])
    ang2 = np.arctan2(*p2[::-1])
    return np.rad2deg((ang1 - ang2) % (2 * np.pi))

In [24]:
def get_Angle(df):
  interactionData= pd.json_normalize(df.interactionData)
  x1=[]
  x2=[]
  y1=[]
  y2=[]
  angle=[]
  for i in range(len(interactionData['interactionType'])):
    #handles if the interactiontype doesn't have a coordinate : if interactionType is zoom
    if ((isinstance(interactionData['state.dragStart'][i], list) or isinstance(interactionData['state.dragEnd'][i], list) )==False):
      
      interactionData['state.dragStart'][i]=[0,0]
      interactionData['state.dragEnd'][i]=[0,0]
    #get between drag start and drag end
    angle.append(angle_between(interactionData['state.dragStart'][i],interactionData['state.dragEnd'][i]))
    #print(angle)

    #this are to plot later on
    x1.append(interactionData['state.dragStart'][i][0])
    y1.append(interactionData['state.dragStart'][i][1])

    x2.append(interactionData['state.dragEnd'][i][0])
    y2.append(interactionData['state.dragEnd'][i][1])
  return angle



In [25]:
#to generate reward based on radiclick and snapshot data 
def get_Reward(dataframe,filenamed):
  #global path
  reward=[]
  subtask=[]
  raw_timestamp= dataframe.timestampMillis
  snapshot_global_drive_path="/content/drive/MyDrive/ForeCache/forecache_user_study_data_dump/forecache_user_study_data_dump"
  #get a pandas dataframe object from json file
  path= snapshot_global_drive_path + "/snapshots.json"
  with open(path) as project_file:    
      data = json.load(project_file) 
  df_snapshots = pd.json_normalize(data)

  #get userid from fileName
  #filenamed[filenamed.find("userid_")+7: filenamed.find(".csv")] need to remove  hard code later
  dataframe_dimension=df_snapshots.loc[(df_snapshots['taskname']=="ndsi-3d-task") & (df_snapshots['userid']==filenamed[filenamed.find("userid_")+7: filenamed.find(".json")])] #change here for different Task name 2d, 3d
  print("There are ", len(dataframe_dimension) , "Snapshot times")
  for i in tqdm(range(len(raw_timestamp))):
      stringmyDate=str(raw_timestamp[i])
      myDate=stringmyDate[: stringmyDate.find(".")]
      dt = datetime.datetime.strptime(myDate, "%Y-%m-%d %H:%M:%S")
      a=time.mktime(dt.timetuple()) + (dt.microsecond / 1000000.0)
      temp=0
      task=1
      for j in range(len(dataframe_dimension["timestampMillis"])):


        if abs(a - (dataframe_dimension.timestampMillis.iloc[j]/1000)) < 10:
          temp=temp+10
        else:
         temp=temp+0
        if (a - (dataframe_dimension.timestampMillis.iloc[j]/1000)) < 0:
          task=1
        else:
          task=2
      subtask.append(task)
      reward.append(temp)
  return reward, subtask
    

In [26]:
def get_Zoom(df_zoom):
  df_zoom['state.newZoom'] = df_zoom['state.newZoom'].replace(np.nan, 0)
  lastZoom=[0,0]
  zoom_level=[]
  for i in tqdm(range(len(df_zoom))):
    if df_zoom['state.newZoom'][i]==0:
      if (lastZoom!=[0,0]):
          df_zoom['state.newZoom'][i]=lastZoom
      else:
        df_zoom['state.newZoom'][i]= [0,0]
        lastZoom= [0,0]
    else:
      lastZoom = df_zoom['state.newZoom'][i]
    zoom_level.append(df_zoom['state.newZoom'][i][0])
  return zoom_level

In [27]:
def get_IdleTime(df):
  idle_time=[]
  previous_time=df['timestampMillis'][0]
  for i in tqdm(range(len(df))):
      wait_time=df['timestampMillis'][i]-previous_time
      previous_time=df['timestampMillis'][i]


      #fixing format
      idle_time.append(wait_time.total_seconds())
  return idle_time

In [28]:
import heapq
def get_Subset_Idletime(idle_time):
  subset_idle_time=[]
  largest_indexes=heapq.nlargest(3, range(len(idle_time)), key=idle_time.__getitem__)
  second_max_index=largest_indexes[2]
  for j in tqdm(range(len(idle_time))):
    print(largest_indexes)
    if (j<= second_max_index):
      subset_idle_time.append(1)
    else:
      subset_idle_time.append(2)
  return subset_idle_time

In [29]:
def get_State(df_state):
  states_array=[]
  for i in range(len(df_state)):
    if (df_state.phase[i]=='Sensemaking'):
      states_array.append(df_state.phase[i])
    else:
      states_array.append("Foraging")
  return states_array

In [30]:
def get_Action(df_action):
  action_array=[]
  last_action=df_action.interactionType[0]
  for i in range(len(df_action)):
    if (df_action.interactionType[i]==last_action):
      action_array.append("keep")
      
    else:
      action_array.append("change")
    last_action=df_action.interactionType[i]
  return action_array

In [15]:
# from collections import defaultdict
# def get_probabilities(df):
#     probabilities=[]
#     map=dict()

#     states=['Sensemaking','Foraging']
#     actions=['start','pan','zoom','radioClick']
#     for state in states:
#       for action in actions:
#         map[state+action]=1


#     for i in range(len(df)):
#       #print(str(df['State'][i]+df['Action']))
#       # if df['State'][i]=="Navigation":
#       #   map[str(df['State'][i]+df['Action'][i])] += 1
#       #   probs = (map[str(df['State'][i]+df['Action'][i])])/(map['Navigationpan'] + map['NavigationradioClick']+ map['Navigationstart'] + map['Navigationzoom'])

#       if df['State'][i]=="Sensemaking":
#         map[str(df['State'][i]+df['Action'][i])] += 1
#         probs = (map[str(df['State'][i]+df['Action'][i])])/(map['Sensemakingpan'] + map['SensemakingradioClick']+ map['Sensemakingstart'] + map['Sensemakingzoom'])
#       else: #df['State'][i]=="Foraging":
#         map[str(df['State'][i]+df['Action'][i])] += 1
#         probs = (map[str(df['State'][i]+df['Action'][i])])/(map['Foragingpan'] + map['ForagingradioClick']+ map['Foragingstart'] + map['Foragingzoom'])
#       # else:
#       #   map[str(df['State'][i]+df['Action'][i])] += 1
#       #   probs = (map[str(df['State'][i]+df['Action'][i])])/(map['Answeringpan'] + map['AnsweringradioClick']+ map['Answeringstart'] + map['Answeringzoom'])
#       probabilities.append(probs)
#     return probabilities

In [31]:
from collections import defaultdict
def get_probabilities(df):
    probabilities=[]
    map=dict()

    states=['Sensemaking','Foraging']
    actions=['change','keep']
    for state in states:
      for action in actions:
        map[state+action]=1


    for i in range(len(df)):
      #print(str(df['State'][i]+df['Action']))
      # if df['State'][i]=="Navigation":
      #   map[str(df['State'][i]+df['Action'][i])] += 1
      #   probs = (map[str(df['State'][i]+df['Action'][i])])/(map['Navigationpan'] + map['NavigationradioClick']+ map['Navigationstart'] + map['Navigationzoom'])

      if df['State'][i]=="Sensemaking":
        map[str(df['State'][i]+df['Action'][i])] += 1
        probs = (map[str(df['State'][i]+df['Action'][i])])/(map['Sensemakingchange'] + map['Sensemakingkeep'])
      else: #df['State'][i]=="Foraging":
        map[str(df['State'][i]+df['Action'][i])] += 1
        probs = (map[str(df['State'][i]+df['Action'][i])])/(map['Foragingchange'] + map['Foragingkeep'])
      # else:
      #   map[str(df['State'][i]+df['Action'][i])] += 1
      #   probs = (map[str(df['State'][i]+df['Action'][i])])/(map['Answeringpan'] + map['AnsweringradioClick']+ map['Answeringstart'] + map['Answeringzoom'])
      probabilities.append(probs)
    return probabilities

MAIN FUNCTION THAT RETURNS THE COMPLETED FILE

In [32]:
error_files=[]

for filename in tqdm(['taskname_ndsi-3d-task_userid_ac5b65d7-f260-4534-9fae-d998b726ed32.json']):
  try:
    #saveDir=(str(global_drive_path) +"/stateActionModel/ndsi-2d")
    saveDir=(str(global_drive_path) +"/stateActionModel/ndsi-3d")
    #saveDir=(str(global_drive_path) +"/stateActionModel/rgb-hurricane-3d")


    #get a pandas dataframe object from json file
    df = pd.read_json(os.path.join(directory,filename))
    print("Read file:",filename)
    interactionData=pd.json_normalize(df.interactionData)
    final_dataframe = pd.DataFrame(columns=['Action','State','TimeStamp','StateActionProbab','IdleTime','Reward','Subtask_Screenshot_Time','Subtask_Idle_Time','ZoomLevel'])
    final_dataframe['Action']=get_Action(interactionData)

    final_dataframe['State']=get_State(df)
    final_dataframe['TimeStamp']=df.timestampMillis
    final_dataframe['IdleTime']=get_IdleTime(df)
    final_dataframe['Subtask_Idle_Time']= get_Subset_Idletime(final_dataframe['IdleTime'])
    final_dataframe['Reward'],final_dataframe['Subtask_Screenshot_Time']= get_Reward(df,filename)
    final_dataframe['Angle']= get_Angle(df)
    final_dataframe['ZoomLevel']= get_Zoom(interactionData)	
    #get probabilities
    final_dataframe['StateActionProbab']=get_probabilities(final_dataframe)
    txt_version= filename.replace("json","csv")
    final_dataframe.to_csv(os.path.join(saveDir,txt_version), index=False)
  except AttributeError:
    print("AttributeError")
    error_files.append(filename)


  0%|          | 0/1 [00:00<?, ?it/s]

Read file: taskname_ndsi-3d-task_userid_ac5b65d7-f260-4534-9fae-d998b726ed32.json



100%|██████████| 178/178 [00:00<00:00, 17162.90it/s]

100%|██████████| 178/178 [00:00<00:00, 19466.17it/s]


[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 177, 109]
[1, 17


100%|██████████| 178/178 [00:00<00:00, 4558.50it/s]

100%|██████████| 1/1 [00:00<00:00,  2.14it/s]


In [33]:
final_dataframe

,Action,State,TimeStamp,StateActionProbab,IdleTime,Reward,Subtask_Screenshot_Time,Subtask_Idle_Time,ZoomLevel,Angle
0,keep,Foraging,2016-06-08 18:11:47.904,0.666667,0.000,0,1,1,0,0.000000
1,change,Foraging,2016-06-08 18:12:29.704,0.500000,41.800,0,1,1,1,0.000000
2,keep,Foraging,2016-06-08 18:12:30.282,0.600000,0.578,0,1,1,2,0.000000
3,change,Sensemaking,2016-06-08 18:12:30.952,0.666667,0.670,0,1,1,2,4.296481
4,keep,Sensemaking,2016-06-08 18:12:31.468,0.500000,0.516,0,1,1,2,2.710408
...,...,...,...,...,...,...,...,...,...,...
173,keep,Sensemaking,2016-06-08 18:15:55.597,0.952941,0.570,0,2,2,2,6.843455
174,keep,Sensemaking,2016-06-08 18:15:56.200,0.953216,0.603,0,2,2,2,9.064860
175,keep,Sensemaking,2016-06-08 18:15:57.074,0.953488,0.874,0,2,2,2,15.973863
176,keep,Sensemaking,2016-06-08 18:15:57.677,0.953757,0.603,0,2,2,2,6.705084


In [16]:
error_files

[]

RANDOM TEST SNIPPETS AFTER THIS POINT

In [ ]:
len(matchingName) # so 19 users are the same 

NameError: ignored

In [ ]:
filenamed="taskname_ndsi-3d-task_userid_8b544d24-3274-4bb0-9719-fd2bccc87b02.csv"

In [ ]:
filenamed[filenamed.find("userid_")+7: filenamed.find(".csv")]

In [ ]:
df.head(5)

In [ ]:
df.phase

In [ ]:
interactionData= pd.io.json.json_normalize(df.interactionData)

In [ ]:
interactionData.interactionType

In [ ]:
final_dataframe = pd.DataFrame(columns=['Action','State','TimeStamp'])
final_dataframe['Action']=interactionData.interactionType
final_dataframe['State']=df.phase
final_dataframe['TimeStamp']=df.timestampMillis

In [ ]:
final_dataframe.head

In [ ]:
final_dataframe['Action']=interactionData.interactionType
final_dataframe['State']=df.phase
final_dataframe['TimeStamp']=df.timestampMillis

In [ ]:
final_dataframe

In [ ]:
for item in df.toFetch:
  print (item)

In [ ]:
toFetch

In [ ]:
import json
from pandas.io.json import json_normalize
normalized_json = json_normalize(df['taskname_ndsi-2d-subtask_userid_3abeecbe-327a-441e-be2a-0dd3763c1d45'])
normalized_json.head(3 )


In [ ]:
normalized_json['toFetch']

In [ ]:
#get a pandas dataframe object from json file
#df = pd.read_json("/content/interactions_data/zoom_0_pos_0_0.json")
import json
import pandas as pd
from pandas.io.json import json_normalize
path= '/content/drive/Shareddrives/Colearning-Exploration/Dataset/forecache_interactions_2016/ndsi_2d_jsons/zoom_0_pos_0_0.json'
with open(path) as project_file:    
    data = json.load(project_file)  

df_zoom = pd.json_normalize(data)
df_zoom.head(5)

In [ ]:
df_zoom.domains